# Stable Diffusion Version 2 - 複数モデル選択型
無料で安全な5つのモデルから選んで画像生成できます

In [ ]:
#@title セル① 必要なライブラリのインストール
!pip install --upgrade diffusers[torch] transformers accelerate -q

In [ ]:
#@title セル③ モデルの選択と読み込み

from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler, DPMSolverMultistepScheduler
from diffusers.models import AutoencoderKL
import torch

# モデル選択
selected_model = "Anything V5 (アニメ)" #@param ["Anything V5 (アニメ)", "Realistic Vision V6.0 (写実的)", "Counterfeit V3.0 (アニメ)", "DreamShaper (万能)", "EpicRealism (写実的)"]

model_info = MODELS[selected_model]
print(f"\n選択されたモデル: {selected_model}")
print(f"説明: {model_info['description']}")
print(f"\nモデルを読み込んでいます...")

# VAEの読み込み
try:
    vae = AutoencoderKL.from_pretrained(model_info['vae'])
except Exception as e:
    print(f"指定されたVAEの読み込みに失敗しました: {e}")
    print("デフォルトVAEを使用します")
    vae = None

# スケジューラーの設定（アニメ系はEuler、リアル系はDPM）
try:
    if "アニメ" in selected_model:
        scheduler = EulerDiscreteScheduler.from_pretrained(model_info['id'], subfolder="scheduler")
    else:
        scheduler = DPMSolverMultistepScheduler.from_pretrained(model_info['id'], subfolder="scheduler")
except Exception as e:
    print(f"カスタムスケジューラーの読み込みに失敗しました: {e}")
    print("モデルのデフォルトスケジューラーを使用します")
    scheduler = None

# パイプラインの作成
try:
    # スケジューラーがある場合は指定、ない場合はモデルのデフォルトを使用
    pipeline_kwargs = {
        "safety_checker": None,
        "requires_safety_checker": False,
        "torch_dtype": torch.float16
    }
    
    if scheduler is not None:
        pipeline_kwargs["scheduler"] = scheduler
    if vae is not None:
        pipeline_kwargs["vae"] = vae
    
    # まずカスタムパイプラインで試す
    try:
        pipe = StableDiffusionPipeline.from_pretrained(
            model_info['id'],
            custom_pipeline="lpw_stable_diffusion",
            **pipeline_kwargs
        )
        print("カスタムパイプライン（lpw_stable_diffusion）を使用")
    except:
        # 標準パイプラインで試す
        pipe = StableDiffusionPipeline.from_pretrained(
            model_info['id'],
            **pipeline_kwargs
        )
        print("標準パイプラインを使用")
        
except Exception as e:
    print(f"エラーが発生しました: {e}")
    raise e

pipe = pipe.to("cuda")
print(f"\n✅ {selected_model}の読み込みが完了しました！")
print(f"\n💡 サンプルプロンプト: {model_info['sample']}")

In [ ]:
#@title セル③ モデルの選択と読み込み

from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler, DPMSolverMultistepScheduler
from diffusers.models import AutoencoderKL
import torch

# モデル選択
selected_model = "Anything V5 (アニメ)" #@param ["Anything V5 (アニメ)", "Realistic Vision V6.0 (写実的)", "Counterfeit V3.0 (アニメ)", "DreamShaper (万能)", "EpicRealism (写実的)"]

model_info = MODELS[selected_model]
print(f"\n選択されたモデル: {selected_model}")
print(f"説明: {model_info['description']}")
print(f"\nモデルを読み込んでいます...")

# VAEの読み込み
vae = AutoencoderKL.from_pretrained(model_info['vae'])

# スケジューラーの設定（アニメ系はEuler、リアル系はDPM）
if "アニメ" in selected_model:
    scheduler = EulerDiscreteScheduler.from_pretrained(model_info['id'], subfolder="scheduler")
else:
    scheduler = DPMSolverMultistepScheduler.from_pretrained(model_info['id'], subfolder="scheduler")

# パイプラインの作成
pipe = StableDiffusionPipeline.from_pretrained(
    model_info['id'],
    scheduler=scheduler,
    vae=vae,
    safety_checker=None,
    requires_safety_checker=False,
    custom_pipeline="lpw_stable_diffusion"
)

pipe = pipe.to("cuda")
print(f"\n✅ {selected_model}の読み込みが完了しました！")
print(f"\n💡 サンプルプロンプト: {model_info['sample']}")

In [ ]:
#@title セル④ 画像生成

import datetime
import os
import random
from IPython.display import display, Image as IPImage

# 出力フォルダの作成
!mkdir -p /content/output

# パラメータ設定
prompt = "masterpiece, best quality, 1girl, japanese school uniform, cherry blossoms, spring, smile" #@param {type:"string"}
negative_prompt = "nsfw, low quality, blurry, bad anatomy" #@param {type:"string"}
num_images = 2 #@param {type:"slider", min:1, max:4, step:1}
width = 512 #@param {type:"slider", min:256, max:1024, step:64}
height = 768 #@param {type:"slider", min:256, max:1024, step:64}
cfg_scale = 7 #@param {type:"slider", min:1, max:20, step:0.5}
steps = 25 #@param {type:"slider", min:10, max:50, step:5}
seed = -1 #@param {type:"number"}

print(f"🎨 画像を生成中...")
print(f"モデル: {selected_model}")
print(f"プロンプト: {prompt}")
print(f"サイズ: {width}x{height}")
print(f"生成枚数: {num_images}枚\n")

generated_files = []

for i in range(num_images):
    # シード値の設定
    if seed == -1:
        current_seed = random.randint(0, 2147483647)
    else:
        current_seed = seed + i
    
    generator = torch.Generator(device="cuda").manual_seed(current_seed)
    
    # 画像生成
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        width=width,
        height=height,
        generator=generator,
        guidance_scale=cfg_scale,
        num_inference_steps=steps
    ).images[0]
    
    # ファイル名の生成
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    model_name = selected_model.split(" ")[0].replace("/", "_")
    filename = f"{model_name}_{timestamp}_{current_seed}.png"
    filepath = f"/content/output/{filename}"
    
    # 画像を保存
    image.save(filepath)
    generated_files.append(filepath)
    
    print(f"✅ 画像 {i+1}/{num_images} を生成しました (seed: {current_seed})")
    
    # 画像を表示
    display(image)

print(f"\n🎉 すべての画像の生成が完了しました！")
print(f"保存先: /content/output/")

In [ ]:
#@title セル⑤ 生成した画像のダウンロード（オプション）

# ZIPファイルとしてダウンロード
!cd /content && zip -r generated_images.zip output/

from google.colab import files
files.download('/content/generated_images.zip')

print("✅ generated_images.zip のダウンロードを開始しました")

## 📝 使い方のヒント

### プロンプトの書き方

**アニメ系モデル（Anything V5, Counterfeit V3.0）**
- 基本構造: `masterpiece, best quality, 1girl/1boy, [特徴], [服装], [背景], [ポーズ/表情]`
- 例: `masterpiece, best quality, 1girl, long hair, school uniform, classroom, smile`

**リアル系モデル（Realistic Vision, EpicRealism）**
- 基本構造: `[撮影スタイル], [被写体], [照明], [構図], [カメラ設定]`
- 例: `professional portrait photography, young woman, natural lighting, close-up, bokeh background`

**万能系モデル（DreamShaper）**
- ファンタジー: `fantasy landscape, floating islands, magical crystals, ethereal atmosphere`
- SF: `cyberpunk city, neon lights, rain, night scene, futuristic`

### 推奨設定

- **CFG Scale**: 
  - アニメ系: 7-10
  - リアル系: 5-8
  - 万能系: 6-9

- **ステップ数**: 
  - 高速生成: 20-25
  - 高品質: 30-40

- **解像度**:
  - ポートレート: 512x768
  - 横長: 768x512
  - 正方形: 512x512 or 768x768